In [10]:
import numpy as np
import pandas as pd
import os
import json

cwd = os.getcwd()

results_path = cwd[:-28] + "Results/T5_unmodified/all_scores_range_0_137_trail_{}.npz"
results_path_sbert = cwd[:-28] + "Results/T5_unmodified/all_sbert_scores_0_137_trail_{}.npz"

results_intent_path = cwd[:-28] + "Results/T5_unmodified/all_scores_range_by_intent_0_137_trail_{}.txt"

intents = ["What drives the economy in this state?",
           "What are the available modes of transport in this state?",
           "How is the government structured in this state?",
           "What about this state's arts and culture attracts you the most?",
           "The main reasons why you would like living in this state"]

sample_size = 2

model = "T5 Unmodifed"

In [11]:

total_data = None
total_data_sbert = []
total_data_intent = {}

for intent in intents:
    total_data_intent[intent] = []

# get all of the "normal" data
for i in range(sample_size):
    temp = np.load(results_path.format(i))
    sbert_temp = np.load(results_path_sbert.format(i))
    data = temp['scores']
    data_sbert = sbert_temp['scores']
    
    if total_data is None:
        # we need to take the mean of axis-one first because each "user-sample" contains a set of predicted summaries. So we first avg those sets, before average across all the data/trials
        total_data = data.mean(axis=1)
        total_data_sbert = data_sbert.mean(axis=1)
    else:
        total_data = np.concatenate((total_data, data.mean(axis=1)), axis=0)
        total_data_sbert = np.concatenate((total_data_sbert, data_sbert.mean(axis=1)), axis=0)

# get all of the "by-intent" data
for i in range(sample_size):
    res = None
    with open(results_intent_path.format(i)) as f:
        res = json.load(f)

    for intent in intents:
        total_data_intent[intent].extend(np.mean(res[intent], axis=1).tolist())

# now calculate the relavent stats
total_mean = np.mean(total_data, axis=0)
total_stddev = np.std(total_data, axis=0)

total_mean_sbert = np.mean(total_data_sbert, axis=0)
total_stddev_sbert = np.std(total_data_sbert, axis=0)

mean_intent = {}
stddev_intent = {}

for intent in intents:
    mean = np.mean(total_data_intent[intent], axis=0)
    stddev = np.std(total_data_intent[intent], axis=0)
    mean_intent[intent] = mean
    stddev_intent[intent] = stddev

metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for i, (row_m, row_s) in enumerate(zip(total_mean, total_stddev)):
    row_str = model + " & "
    for col_m, col_s in zip(row_m, row_s):
        row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
    
    row_str = row_str[:-3]
    row_str += " \\\\"
    print(metrics[i])
    print(row_str)
print("SBERT Cosime Sim. Score: ")
print(model + " & " + str(np.round(total_mean_sbert, 4)) + " $\\pm$ " + str(np.round(total_stddev_sbert, 4)) + " \\\\")

print("\n\n\n")

Rouge-1
T5 Unmodifed & 0.1253 $\pm$ 0.0348 & 0.494 $\pm$ 0.0819 & 0.1928 $\pm$ 0.0427 & 0.1205 $\pm$ 0.0267 \\
Rouge-2
T5 Unmodifed & 0.0292 $\pm$ 0.0166 & 0.1264 $\pm$ 0.081 & 0.0459 $\pm$ 0.0263 & 0.0287 $\pm$ 0.0164 \\
Rouge-L
T5 Unmodifed & 0.0871 $\pm$ 0.0225 & 0.3507 $\pm$ 0.0679 & 0.1347 $\pm$ 0.0279 & 0.0842 $\pm$ 0.0175 \\
SBERT Cosime Sim. Score: 
T5 Unmodifed & 0.4472 $\pm$ 0.0784 \\






In [12]:
metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for intent in intents:
    print("\nResults for intent {}\n".format(intent))
    for i, (row_m, row_s) in enumerate(zip(mean_intent[intent], stddev_intent[intent])):
        row_str = "\t"+model + " & "
        for col_m, col_s in zip(row_m, row_s):
            row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
        
        row_str = row_str[:-3]
        row_str += " \\\\"
        print("\t"+metrics[i])
        print(row_str)


Results for intent What drives the economy in this state?

	Rouge-1
	T5 Unmodifed & 0.125 $\pm$ 0.0339 & 0.4833 $\pm$ 0.0704 & 0.1928 $\pm$ 0.0422 & 0.1205 $\pm$ 0.0264 \\
	Rouge-2
	T5 Unmodifed & 0.0298 $\pm$ 0.0187 & 0.1209 $\pm$ 0.0651 & 0.0464 $\pm$ 0.0278 & 0.029 $\pm$ 0.0174 \\
	Rouge-L
	T5 Unmodifed & 0.0879 $\pm$ 0.023 & 0.3462 $\pm$ 0.0574 & 0.136 $\pm$ 0.0289 & 0.085 $\pm$ 0.018 \\

Results for intent What are the available modes of transport in this state?

	Rouge-1
	T5 Unmodifed & 0.127 $\pm$ 0.0326 & 0.479 $\pm$ 0.0603 & 0.1941 $\pm$ 0.0382 & 0.1213 $\pm$ 0.0239 \\
	Rouge-2
	T5 Unmodifed & 0.0228 $\pm$ 0.0072 & 0.0934 $\pm$ 0.0311 & 0.0356 $\pm$ 0.0103 & 0.0223 $\pm$ 0.0065 \\
	Rouge-L
	T5 Unmodifed & 0.0857 $\pm$ 0.0202 & 0.3284 $\pm$ 0.0444 & 0.1313 $\pm$ 0.0232 & 0.082 $\pm$ 0.0145 \\

Results for intent How is the government structured in this state?

	Rouge-1
	T5 Unmodifed & 0.1271 $\pm$ 0.0406 & 0.467 $\pm$ 0.0607 & 0.1905 $\pm$ 0.0478 & 0.1191 $\pm$ 0.0299 \\
	Roug